# Analyse ambigous named entities

One specific token can have different labels in the context of named entity recognition. This experiment allows to read in all tokens and their labels and change the tokens' labels. E.g. if the token <code>Kiidjerwe</code> has the labels <code>LOC</code> and <code>ORG</code>, we may want to change all `ORG` labels to `LOC` if the token has the `ORG` label more than 95% of the times it's in the corpus.

In [20]:
import os
import math
import json

from estnltk import Text
from estnltk.converters import json_to_text
from estnltk.converters import text_to_json
from modules.results_extraction import extract_results, results_by_subdistribution

from estnltk.layer.layer import Layer
from estnltk import EnvelopingBaseSpan
from estnltk.layer_operations import flatten

from nervaluate import Evaluator
import pandas as pd

from modules.preprocessing_protocols import preprocess_text

Divided corpus file for filenames and layers to be removed later on:

In [2]:
divided_corpus = os.path.join('..', 'data', 'divided_corpus.txt')
no_goldstandard_tags_location = os.path.join('..', 'data', 'files_without_goldstandard_annotations.txt')
testing_files_location = os.path.join('..', 'data', 'vallakohtufailid-json-flattened')

removed_layers = ['compound_tokens', 'tokens', 'words']

Read files from <code>path_to_read</code> and write files to <code>path_to_write</code>

In [3]:
path_to_read = os.path.join('models',
                    'model_morph_with_lemmas_and_sentences_and_gazetteer_and_global_features', 
                    'model_gaz_loc_variants', 
                    'vallakohtufailid-trained-nertagger')

path_to_write = os.path.join('models',
                    'model_morph_with_lemmas_and_sentences_and_gazetteer_and_global_features', 
                    'model_gaz_loc_variants_me', 
                    'vallakohtufailid-trained-nertagger')

Get files without goldstandard annotations:

In [4]:
with open(no_goldstandard_tags_location, 'r', encoding='UTF-8') as in_f:
    lines = in_f.readlines()

no_goldstandard_annotations = [line.strip() for line in lines]

In [5]:
files = {}

with open(divided_corpus, 'r', encoding = 'UTF-8') as f:
    txt = f.readlines()

for filename in txt:
    file, subdistribution = filename.split(':')
    files[file] = subdistribution.strip()

In [6]:
def results_by_subdistribution_me(results_json):
    correct = results_json[setup]['strict']['correct']
    actual = results_json[setup]['strict']['actual']
    possible = results_json[setup]['strict']['possible']

    precision = (correct / actual)
    recall = (correct / possible)
    f1 = 2 * ((precision * recall) / (precision + recall))

    data = {'Precision': precision, 'Recall': recall, 'F1': f1}
    results_df = pd.DataFrame(data, index=[0])
    
    return data

In [7]:
def multivalued_entity_dict(path, leave_out_O):
    me_dict = dict()
    texts_dict = dict()
    
    # Read all filenames from path
    for file in os.listdir(path):
        with open(os.path.join(path, file), 'r', encoding='UTF-8') as in_f:
            texts_dict[file] = json_to_text(in_f.read())
    
    # Create me_dict, which contains all named entities
    for file in texts_dict.keys():
        text = texts_dict[file]
        
        for entity in text.flat_wordner:
            named_entity = entity.text
            nertag = entity.nertag[0]
            labels = dict()
            if entity.text in me_dict.keys():
                if nertag in me_dict[named_entity]:
                    labels = me_dict[named_entity]
                    labels[nertag] = labels[nertag] + 1
                else:
                    labels = me_dict[named_entity]
                    labels[nertag] = 1
                    me_dict[named_entity] = labels
            else:
                me_dict[named_entity] = {nertag: 1}

    # If leave_out_O == True, leave out all named entites that
    # have other labels as well as the 'O' label 
    delete_keys = set()
    for key, value in me_dict.items():
        if len(value.items()) == 1:
            delete_keys.add(key)
        elif leave_out_O and 'O' in value.keys():
            delete_keys.add(key)

    for key in delete_keys:
        del me_dict[key]

    for key, value in me_dict.items():
        all_counts = 0
        for ne, count in value.items():
            all_counts += count

        for ne, count in value.items():
            percentage = round((count / all_counts) * 100, 3)
            value[ne] = percentage

    return me_dict

In [8]:
def change_multivalued_entities_in_file(text, me_dict, threshold, threshold_O = 100):
    
    for i in range(len(text.flat_wordner)):
        entity = text.flat_wordner[i].text
        nertag = text.flat_wordner[i].nertag[0]
        
        # If the named entity is in the dict, check if it needs to be
        # changed due to the threshold
        
        if entity in me_dict:
            for key, value in me_dict[entity].items():
                before = text.flat_wordner[i].nertag[0]
                if key != 'O' and value > threshold and nertag != key:
                    text.flat_wordner[i].nertag = key
                    print(f'Muudatus sõnes {entity}: {before} -> {key}')
                elif key == 'O' and value > threshold_O and nertag != key:
                    text.flat_wordner[i].nertag = key
                    print(f'Muudatus sõnes {entity}: {before} -> {key}')
    return text

The `wordner` layer is converted to `ner` layer due to the results extraction being built upon `ner` tags and not `wordner` tags.

In [9]:
def convert_wordner_to_ner(text):
    snt_labels = [label.nertag[0] for label in text.flat_wordner]
    
    text.pop_layer('flat_ner')
    text = preprocess_text(text)    
    nerlayer = Layer(name='ner', attributes=['nertag'], text_object=text, enveloping='words')
    
    entity_spans = []
    entity_type = None
    
    for span, label in zip(text.words, snt_labels):
        if entity_type is None:
            entity_type = label[2:]
        if label == "O":
            if entity_spans:
                nerlayer.add_annotation(EnvelopingBaseSpan(entity_spans),
                                        **{['nertag'][0]: entity_type})
                entity_spans = []
            continue
        if label[0] == "B" or entity_type != label[2:]:
            if entity_spans:
                nerlayer.add_annotation(EnvelopingBaseSpan(entity_spans),
                                        **{['nertag'][0]: entity_type})
                entity_spans = []
        entity_type = label[2:]
        entity_spans.append(span.base_span)
        
    text.add_layer(nerlayer)
    
    flat_ner = flatten(text['ner'], 'flat_ner')
    text.add_layer(flat_ner)
    
    for layer in removed_layers:
        text.pop_layer(layer)
    return text

---

In [10]:
setups = [[True, 85], # Leave out O-s, if majority above 85% -> change
          [True, 90], # Leave out O-s, if majority above 90% -> change 
          [True, 95], # Leave out O-s, if majority above 95% -> change
          [False, 85, 95], # Don't leave out O-s, if majority of !O above 85% -> change, also change O if above 95%
          [False, 85, 99], # Don't leave out O-s, if majority of !O above 85% -> change, also change O if above 99%
          [False, 90, 95], # Don't leave out O-s, if majority of !O above 90% -> change, also change O if above 95%
          [False, 90, 99], # Don't leave out O-s, if majority of !O above 90% -> change, also change O if above 99%
          [False, 95, 95], # Don't leave out O-s, if majority of !O above 95% -> change, also change O if above 95%
          [False, 95, 99]] # Don't leave out O-s, if majority of !O above 95% -> change, also change O if above 99%

In [11]:
all_results = dict()

for setup in setups:
    leave_out_O = setup[0]
    threshold = int(setup[1])
    try:
        threshold_O = setup[2]
    except:
        threshold_O = 100
        
    me_dict = multivalued_entity_dict(path_to_read, leave_out_O)
    
    for file in os.listdir(path_to_read):
        if file.endswith('json'):
            with open(os.path.join(path_to_read, file), 'r', encoding='UTF-8') as in_f:
                text = json_to_text(in_f.read())
            
            text = change_multivalued_entities_in_file(text, me_dict, threshold, threshold_O)
            text = convert_wordner_to_ner(text)
            text_to_json(text, file=os.path.join(path_to_write, file))
    setup_string = f'{str(leave_out_O)}, {str(threshold)}, {str(threshold_O)}'
    
    extract_results(files, no_goldstandard_annotations, path_to_write, testing_files_location, path_to_write)
    
    with open(os.path.join(path_to_write, 'results.txt'), 'r', encoding='UTF-8') as in_f:
        all_results[setup_string] = json.loads(in_f.read())
    
with open(os.path.join(path_to_write, 'results.txt'), 'w+', encoding='UTF-8') as out_f:
    json.dump(all_results, out_f)

Muudatus sõnes Jakub: B-LOC_ORG -> B-PER
Muudatus sõnes Hindrek: I-PER -> B-PER
Muudatus sõnes Hindrek: I-PER -> B-PER
Muudatus sõnes Hindrek: I-PER -> B-PER
Muudatus sõnes Jaan: I-PER -> B-PER
Muudatus sõnes Lauk: B-PER -> I-PER
Muudatus sõnes Michelsoni: B-PER -> I-PER
Muudatus sõnes Isak: I-PER -> B-PER
Muudatus sõnes Ano: B-LOC_ORG -> B-PER
Muudatus sõnes Willewelt: B-PER -> I-PER
Muudatus sõnes Noarootsi: B-ORG -> I-ORG
Muudatus sõnes Jürrij: I-PER -> B-PER
Muudatus sõnes Jürrij: I-PER -> B-PER
Muudatus sõnes Tannil: I-PER -> B-PER
Muudatus sõnes Tannil: I-PER -> B-PER
Muudatus sõnes Jahn: I-PER -> B-PER
Muudatus sõnes Anno: B-LOC_ORG -> B-PER
Muudatus sõnes Käit: B-PER -> I-PER
Muudatus sõnes Iwanow: B-PER -> I-PER
Muudatus sõnes Tomkin: B-PER -> I-PER
Muudatus sõnes Nagel: B-PER -> I-PER
Muudatus sõnes Johann: I-PER -> B-PER
Muudatus sõnes Lobba: B-LOC_ORG -> I-PER
Muudatus sõnes Sillaberg: B-PER -> I-PER
Muudatus sõnes F: I-PER -> B-PER
Muudatus sõnes Aadu: I-PER -> B-PER
Muuda

Muudatus sõnes Jaan: I-PER -> B-PER
Muudatus sõnes Lauk: B-PER -> I-PER
Muudatus sõnes Michelsoni: B-PER -> I-PER
Muudatus sõnes Jürrij: I-PER -> B-PER
Muudatus sõnes Jürrij: I-PER -> B-PER
Muudatus sõnes Tannil: I-PER -> B-PER
Muudatus sõnes Tannil: I-PER -> B-PER
Muudatus sõnes Jahn: I-PER -> B-PER
Muudatus sõnes Anno: B-LOC_ORG -> B-PER
Muudatus sõnes Iwanow: B-PER -> I-PER
Muudatus sõnes Nagel: B-PER -> I-PER
Muudatus sõnes Johann: I-PER -> B-PER
Muudatus sõnes Sillaberg: B-PER -> I-PER
Muudatus sõnes Pastak: B-LOC_ORG -> I-PER
Muudatus sõnes Jaak: I-PER -> B-PER
Muudatus sõnes Jaak: I-PER -> B-PER
Muudatus sõnes Jaak: I-PER -> B-PER
Muudatus sõnes Jakob: I-PER -> B-PER
Muudatus sõnes Jaak: I-PER -> B-PER
Muudatus sõnes Miina: I-PER -> B-PER
Muudatus sõnes Jaan: I-PER -> B-PER
Muudatus sõnes Märt: I-PER -> B-PER
Muudatus sõnes Jaan: I-PER -> B-PER
Muudatus sõnes Koor: B-PER -> I-PER
Muudatus sõnes Matsoni: B-PER -> I-PER
Muudatus sõnes Ilwes: B-PER -> I-PER
Muudatus sõnes Mats: I-P

Muudatus sõnes P: I-PER -> B-PER
Muudatus sõnes P: I-PER -> B-PER
Muudatus sõnes P: I-PER -> B-PER
Muudatus sõnes P: I-PER -> B-PER
Muudatus sõnes P: I-PER -> B-PER
Muudatus sõnes P: I-PER -> B-PER
Muudatus sõnes Jakub: B-LOC_ORG -> B-PER
Muudatus sõnes P: I-PER -> B-PER
Muudatus sõnes P: I-PER -> B-PER
Muudatus sõnes ?: I-PER -> O
Muudatus sõnes P: I-PER -> B-PER
Muudatus sõnes P: I-PER -> B-PER
Muudatus sõnes Jüri: B-LOC_ORG -> B-PER
Muudatus sõnes Hindrek: I-PER -> B-PER
Muudatus sõnes Hindrek: I-PER -> B-PER
Muudatus sõnes Hindrek: I-PER -> B-PER
Muudatus sõnes Madli: B-LOC_ORG -> B-PER
Muudatus sõnes Madli: O -> B-PER
Muudatus sõnes metsa: I-LOC -> O
Muudatus sõnes Jaan: I-PER -> B-PER
Muudatus sõnes Lauk: B-PER -> I-PER
Muudatus sõnes Selle: I-PER -> O
Muudatus sõnes Michelsoni: B-PER -> I-PER
Muudatus sõnes :: I-PER -> O
Muudatus sõnes :: I-PER -> O
Muudatus sõnes :: I-PER -> O
Muudatus sõnes :: I-PER -> O
Muudatus sõnes ja: I-ORG -> O
Muudatus sõnes Isak: I-PER -> B-PER
Muudatu

Muudatus sõnes Jüri: I-PER -> B-PER
Muudatus sõnes k: I-PER -> O
Muudatus sõnes Mik: I-PER -> B-PER
Muudatus sõnes Mik: I-PER -> B-PER
Muudatus sõnes Nikke: O -> I-PER
Muudatus sõnes ja: I-ORG -> O
Muudatus sõnes Karli: I-PER -> B-PER
Muudatus sõnes a: I-PER -> O
Muudatus sõnes Tikk: B-PER -> I-PER
Muudatus sõnes [: I-PER -> O
Muudatus sõnes -: I-LOC -> O
Muudatus sõnes -: I-LOC -> O
Muudatus sõnes -: I-LOC -> O
Muudatus sõnes -: I-LOC_ORG -> O
Muudatus sõnes -: I-PER -> O
Muudatus sõnes -: I-LOC_ORG -> O
Muudatus sõnes -: I-LOC -> O
Muudatus sõnes -: I-LOC_ORG -> O
Muudatus sõnes -: I-LOC_ORG -> O
Muudatus sõnes -: I-LOC_ORG -> O
Muudatus sõnes -: I-LOC -> O
Muudatus sõnes -: I-LOC_ORG -> O
Muudatus sõnes Pedoson: B-PER -> I-PER
Muudatus sõnes Reisenpuk: B-PER -> I-PER
Muudatus sõnes ja: I-ORG -> O
Muudatus sõnes Jüri: I-ORG -> B-PER
Muudatus sõnes Jüri: O -> B-PER
Muudatus sõnes kohto: I-ORG -> O
Muudatus sõnes -: I-PER -> O
Muudatus sõnes Lentsmani: I-LOC -> I-PER
Muudatus sõnes ja:

Muudatus sõnes K: O -> B-PER
Muudatus sõnes K: O -> B-PER
Muudatus sõnes H: O -> B-PER
Muudatus sõnes K: O -> B-PER
Muudatus sõnes Jüri: O -> B-PER
Muudatus sõnes Juri: I-PER -> B-PER
Muudatus sõnes Reisenpuk: O -> I-PER
Muudatus sõnes Karli: B-LOC_ORG -> B-PER
Muudatus sõnes Jaan: I-PER -> B-PER
Muudatus sõnes Jürri: O -> B-PER
Muudatus sõnes Jürri: B-LOC_ORG -> B-PER
Muudatus sõnes Epp: I-PER -> B-PER
Muudatus sõnes ja: I-ORG -> O
Muudatus sõnes Mikson: B-PER -> I-PER
Muudatus sõnes Daniel: I-PER -> B-PER
Muudatus sõnes Jürri: I-PER -> B-PER
Muudatus sõnes Kristjaan: B-LOC_ORG -> B-PER
Muudatus sõnes wallast: O -> I-LOC_ORG
Muudatus sõnes aial: B-PER -> O
Muudatus sõnes Lokko: B-PER -> I-PER
Muudatus sõnes Maie: O -> B-PER
Muudatus sõnes Redik: I-PER -> B-PER
Muudatus sõnes ja: I-ORG -> O
Muudatus sõnes ja: I-ORG -> O
Muudatus sõnes Jüri: O -> B-PER
Muudatus sõnes Kusta: I-PER -> B-PER
Muudatus sõnes Andrese: I-PER -> B-PER
Muudatus sõnes -: I-LOC -> O
Muudatus sõnes Kusta: I-PER -> 

...

Muudatus sõnes -: I-PER -> O
Muudatus sõnes -: I-LOC_ORG -> O
Muudatus sõnes -: I-PER -> O
Muudatus sõnes -: I-ORG -> O
Muudatus sõnes -: I-LOC_ORG -> O
Muudatus sõnes Härg: O -> I-PER
Muudatus sõnes ja: I-ORG -> O
Muudatus sõnes Kohtu: I-ORG -> O
Muudatus sõnes Tomson: B-PER -> I-PER
Muudatus sõnes Jürri: O -> B-PER
Muudatus sõnes Jürri: O -> B-PER
Muudatus sõnes Aid: O -> I-PER
Muudatus sõnes Jüri: O -> B-PER
Muudatus sõnes -: I-LOC -> O
Muudatus sõnes Nõmm: B-PER -> I-PER
Muudatus sõnes Kohtu: I-ORG -> O
Muudatus sõnes Otto: I-PER -> B-PER
Muudatus sõnes Tomson: B-PER -> I-PER
Muudatus sõnes Peet: I-PER -> B-PER
Muudatus sõnes kohto: I-ORG -> O
Muudatus sõnes P: O -> B-PER
Muudatus sõnes Ernits: B-PER -> I-PER
Muudatus sõnes Anu: B-LOC_ORG -> B-PER
Muudatus sõnes Pehk: B-PER -> I-PER
Muudatus sõnes Ado: I-PER -> B-PER
Muudatus sõnes Jüri: I-PER -> B-PER
Muudatus sõnes ja: I-ORG -> O
Muudatus sõnes ja: I-ORG -> O
Muudatus sõnes Tannil: B-LOC_ORG -> B-PER
Muudatus sõnes Piir: B-PER ->

Muudatus sõnes Jüri: O -> B-PER
Muudatus sõnes Hind: O -> B-PER
Muudatus sõnes Jürri: O -> B-PER
Muudatus sõnes Jürri: O -> B-PER
Muudatus sõnes XXX: B-PER -> O
Muudatus sõnes XXX: B-PER -> O
Muudatus sõnes XXX: B-PER -> O
Muudatus sõnes XXX: B-PER -> O
Muudatus sõnes XXX: B-PER -> O
Muudatus sõnes XXX: B-PER -> O
Muudatus sõnes XXX: B-PER -> O
Muudatus sõnes ja: I-LOC -> O
Muudatus sõnes Jacob: I-PER -> B-PER
Muudatus sõnes Matsoni: B-PER -> I-PER
Muudatus sõnes ja: I-ORG -> O
Muudatus sõnes Wenne: O -> I-PER
Muudatus sõnes ja: I-LOC -> O
Muudatus sõnes Weber: O -> I-PER
Muudatus sõnes Ilwes: B-PER -> I-PER
Muudatus sõnes :: I-PER -> O
Muudatus sõnes Jürri: O -> B-PER
Muudatus sõnes :: I-PER -> O
Muudatus sõnes :: I-PER -> O
Muudatus sõnes :: I-PER -> O
Muudatus sõnes :: I-PER -> O
Muudatus sõnes :: I-PER -> O
Muudatus sõnes :: I-PER -> O
Muudatus sõnes :: I-PER -> O
Muudatus sõnes Tanil: O -> B-PER
Muudatus sõnes wallas: O -> I-LOC_ORG
Muudatus sõnes Mats: I-PER -> B-PER
Muudatus sõn

Muudatus sõnes Kask: O -> I-PER
Muudatus sõnes Wirk: B-PER -> I-PER
Muudatus sõnes Mitt: B-PER -> I-PER
Muudatus sõnes ,: I-PER -> O
Muudatus sõnes P: O -> B-PER
Muudatus sõnes A: O -> B-PER
Muudatus sõnes Jürgenson: B-PER -> I-PER
Muudatus sõnes Krimm: B-PER -> I-PER
Muudatus sõnes Jürri: O -> B-PER
Muudatus sõnes Abram: I-PER -> B-PER
Muudatus sõnes Leht: O -> I-PER
Muudatus sõnes Tuhha: O -> I-PER
Muudatus sõnes Josepi: O -> B-PER
Muudatus sõnes ): I-LOC_ORG -> O
Muudatus sõnes Jahn: I-PER -> B-PER
Muudatus sõnes G: O -> B-PER
Muudatus sõnes Kikkas: B-LOC_ORG -> I-PER
Results have been saved to models/model_morph_with_lemmas_and_sentences_and_gazetteer_and_global_features/model_gaz_loc_variants_me/vallakohtufailid-trained-nertagger/results.txt
Muudatus sõnes Jakub: B-LOC_ORG -> B-PER
Muudatus sõnes ?: I-PER -> O
Muudatus sõnes Hindrek: I-PER -> B-PER
Muudatus sõnes Hindrek: I-PER -> B-PER
Muudatus sõnes Hindrek: I-PER -> B-PER
Muudatus sõnes Madli: B-LOC_ORG -> B-PER
Muudatus sõnes 

Muudatus sõnes Anne: I-PER -> B-PER
Muudatus sõnes Edro: B-PER -> I-PER
Muudatus sõnes Jürri: O -> B-PER
Muudatus sõnes Jürri: O -> B-PER
Muudatus sõnes ja: I-ORG -> O
Muudatus sõnes toas: I-LOC -> O
Muudatus sõnes ja: I-LOC -> O
Muudatus sõnes Jakob: I-PER -> B-PER
Muudatus sõnes Kohto: I-LOC -> O
Muudatus sõnes :: I-PER -> O
Muudatus sõnes :: I-PER -> O
Muudatus sõnes :: I-PER -> O
Muudatus sõnes kohto: I-ORG -> O
Muudatus sõnes ): I-LOC_ORG -> O
Muudatus sõnes -: I-PER -> O
Muudatus sõnes -: I-PER -> O
Muudatus sõnes -: I-PER -> O
Muudatus sõnes ja: I-LOC -> O
Muudatus sõnes kohto: I-ORG -> O
Muudatus sõnes A: I-MISC -> B-PER
Muudatus sõnes :: I-PER -> O
Muudatus sõnes :: I-PER -> O
Muudatus sõnes Ado: I-PER -> B-PER
Muudatus sõnes tütar: I-PER -> O
Muudatus sõnes ja: I-LOC -> O
Muudatus sõnes -: I-PER -> O
Muudatus sõnes -: I-LOC_ORG -> O
Muudatus sõnes -: I-PER -> O
Muudatus sõnes -: I-ORG -> O
Muudatus sõnes -: I-LOC_ORG -> O
Muudatus sõnes Härg: O -> I-PER
Muudatus sõnes ja: I-O

Muudatus sõnes Tõnnis: I-PER -> B-PER
Muudatus sõnes Tõnnis: I-PER -> B-PER
Muudatus sõnes ja: I-LOC -> O
Muudatus sõnes Marri: I-PER -> B-PER
Muudatus sõnes ,: I-PER -> O
Muudatus sõnes Jaan: I-PER -> B-PER
Muudatus sõnes Jaan: I-PER -> B-PER
Muudatus sõnes Jaan: I-PER -> B-PER
Muudatus sõnes Jaan: I-PER -> B-PER
Muudatus sõnes 9: I-LOC_ORG -> O
Muudatus sõnes Kadri: I-PER -> B-PER
Muudatus sõnes Liso: I-PER -> B-PER
Muudatus sõnes Purge: B-PER -> I-PER
Muudatus sõnes Hindrikson: B-PER -> I-PER
Muudatus sõnes Anne: I-PER -> B-PER
Muudatus sõnes Edro: B-PER -> I-PER
Muudatus sõnes Jürri: O -> B-PER
Muudatus sõnes Jürri: O -> B-PER
Muudatus sõnes ja: I-ORG -> O
Muudatus sõnes ja: I-LOC -> O
Muudatus sõnes Jakob: I-PER -> B-PER
Muudatus sõnes Kohto: I-LOC -> O
Muudatus sõnes :: I-PER -> O
Muudatus sõnes :: I-PER -> O
Muudatus sõnes :: I-PER -> O
Muudatus sõnes ): I-LOC_ORG -> O
Muudatus sõnes ja: I-LOC -> O
Muudatus sõnes A: I-MISC -> B-PER
Muudatus sõnes :: I-PER -> O
Muudatus sõnes :: 

---

In [17]:
with open(os.path.join(path_to_write, 'results.txt'), 'r', encoding='UTF-8') as in_f:
    data = json.loads(in_f.read())

In [46]:
ending_df = dict()

for key in data.keys():
    df = pd.DataFrame(results_by_subdistribution(data[key], files))
    p = df['Total'].Precision
    r = df['Total'].Recall
    f = df['Total'].F1
    
    ending_df[key] = {'Täpsus': p, 'Saagis': r, 'F1': f}

In [47]:
display(pd.DataFrame(ending_df))

,"True, 85, 100","True, 90, 100","True, 95, 100","False, 85, 95","False, 85, 99","False, 90, 95","False, 90, 99","False, 95, 95","False, 95, 99"
Täpsus,0.905425,0.906599,0.906827,0.889608,0.895018,0.893169,0.898741,0.894792,0.900389
Saagis,0.876786,0.877301,0.876743,0.873267,0.875928,0.873696,0.876486,0.872366,0.875156
F1,0.890875,0.891710,0.891532,0.881362,0.885370,0.883325,0.887474,0.883437,0.887593


The first `boolean` value states if `O` tags are completely left out or not.

The second value states the threshold for changing named entity labels from one to another:
e.g. if "Jaan" is 91% `B-PER` and 9% `I-PER` and if the threshold is `90`, all the `I-PER` tags
are also changed to `B-PER`. Should the threshold be for example 92, the named entity label 
stays unchanged.

The third value states the threshold for changing labels from or to `O` based on the threshold.